In [4]:
import py3Dmol
import numpy as np
import os

import pandas as pd

def write_configs(dock_name, receptor, ligand, aa_list, centers, x_size, y_size, z_size):
    # function to write the configs file for AutoDock VINA
    # write_configs(dock_name, receptor, ligand, aa_list, centers, x_size, y_size, z_size)
    #   dock_name (str) = unique name of ligand + receptor complex
    #   receptor (str) = name of receptor pdbqt file
    #   ligand (str) = name of ligand pdbqt file
    #   aa_list (list) = list of flexible residues numbers in the receptor
    #   centers (list) = list of x,y,x centers of docking site in A
    #   x_size, y_size, z_size (int) = size of docking grids in x,y,z dimensions (A)
    
    with open(f"{dock_name}_config_singledock","w") as f:
      f.write("#CONFIGURATION FILE (options not used are commented) \n")
      f.write("\n")
      f.write("#INPUT OPTIONS \n")
      f.write(f"receptor = {receptor}.pdbqt \n")
      f.write(f"ligand = {ligand}.pdbqt \n")
      f.write(f"#flex = {aa_list} \n")
      f.write("#SEARCH SPACE CONFIGURATIONS \n")
      f.write("#Center of the box (values bxi, byi and bzi) \n")
 
      f.write(f"center_x = {centers[0]} \n")
      f.write(f"center_y = {centers[1]} \n")
      f.write(f"center_z = {centers[2]} \n")
    
      f.write("#Size of the box (values bxf, byf and bzf) \n")
      f.write(f"size_x = {x_size} \n")
      f.write(f"size_y = {y_size} \n")
      f.write(f"size_z = {z_size} \n")
      f.write("#OUTPUT OPTIONS \n")
      f.write("#out = \n")
      f.write("#log = \n")
      f.write("\n")
      f.write("#OTHER OPTIONS \n")
      f.write("#cpu =  \n")
      f.write("#exhaustiveness = \n")
      f.write("#num_modes = \n")
      f.write("#energy_range = \n")
      f.write("#seed = ")

def take_loss1(filename, aminoacids):
  # function for defining the receptor docking center based on flexible residues
  # take_loss1(filename, aminoacids):
  #   filename (str) = name of receptor pdb file
  #   aminoacids (list) = list of flexible residues numbers in the receptor
    
  with open(filename) as ifile:
      		system = "".join([x for x in ifile])
  system1 = system.split("\n")
  system2 = []
  for x in system1:
    		if x[:4] == 'ATOM':
      			system2.append(x)
  CAS = [x for x in system2 if "CA" in x]
  CAS = [x.split(' ') for x in CAS]
  CAS = [[x for x in y if x!=''] for y in CAS ]
  CAS_A = [x for x in CAS if x[4]=='A']


  CAS_A = {str(int(x[5])): np.array([float(x[6]), float(x[7]), float(x[8])]) for x in CAS_A}

  CAS_A = np.array([CAS_A[x] for x in aminoacids])

  receptor_center = (sum([i for i in CAS_A]))/CAS_A.shape[0]
    
  return receptor_center

In [7]:
# DEFINE LIGANDS
# creating smiles representations of ligands for docking

ATP = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])O)O)O)N'
ADP = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])O)O)O)N'
AMP = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])O)O)O)N'
adenosine = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO)O)O)N'
diadenosine_polyp_2 = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])O[C@@H]4[C@H](O[C@@H]([C@@H]4O)N5C=NC6=C(N=CN=C65)N)CO)O)O)N'
diadenosine_polyp_3 = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OC[C@@H]4[C@H]([C@H]([C@@H](O4)N5C=NC6=C(N=CN=C65)N)O)O)O)O)N'
diadenosine_polyp_4 = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OC[C@@H]4[C@H]([C@H]([C@@H](O4)N5C=NC6=C(N=CN=C65)N)O)O)O)O)N'
diadenosine_polyp_5 = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OC[C@@H]4[C@H]([C@H]([C@@H](O4)N5C=NC6=C(N=CN=C65)N)O)O)O)O)N'
diadenosine_polyp_6 = 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OP(=O)([O-])OC[C@@H]4[C@H]([C@H]([C@@H](O4)N5C=NC6=C(N=CN=C65)N)O)O)O)O)N'
polyP_14 = '[O-]'+'P(=O)([O-])O'*13+'P(=O)([O-])[O-]'
cpolyP_14 = 'O1'+'P(=O)([O-])O'*13+'P(=O)([O-])1'
cpolyP_6 = 'O1'+'P(=O)([O-])O'*5+'P(=O)([O-])1'
polyP_40 = '[O-]'+'P(=O)([O-])O'*39+'P(=O)([O-])[O-]'
polyP_60 = '[O-]'+'P(=O)([O-])O'*59+'P(=O)([O-])[O-]'
polyP_100 = '[O-]'+'P(=O)([O-])O'*99+'P(=O)([O-])[O-]'
UTP = 'C1=CN(C(=O)NC1=O)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])O)O)O'
UDP = 'C1=CN(C(=O)NC1=O)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)([O-])OP(=O)([O-])O)O)O'
UDP_glucose = 'C1=CN(C(=O)NC1=O)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)([O-])OP(=O)([O-])O[C@@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)O)O'

In [ ]:
# RECEPTOR AND LIGAND PREPARATION

# selection of flexible residues in receptor
p2y1 = [40, 41, 43, 44, 45, 46, 110, 115, 195, 196, 201, 203, 204, 205, 206, 207, 208, 283, 287, 303, 306, 307, 310] #flecible residues list
p2y2 = [26, 27, 29, 32, 92, 93, 97, 113, 114, 118, 177, 180, 182, 184, 185, 186, 187, 189, 192, 195, 264, 267, 268, 271, 284, 287, 288, 291]
p2y4 = [22, 25, 27, 30, 31, 38, 90, 91, 94, 95, 96, 108, 111, 112, 115, 116, 175, 178, 180, 182, 183, 184, 190, 193, 257, 258, 261, 264, 265, 268, 281, 284, 285, 288]
p2y6 = [19, 20, 22, 25, 33, 75, 86, 90, 103, 106, 107, 171, 172, 174, 178, 179, 180, 183, 255, 256, 259, 262, 266, 283, 284, 287]
p2y12 = [25, 27, 38, 49, 83, 86, 110, 111, 179, 180, 182, 183, 193, 258, 262, 265, 269, 279, 283, 286, 287, 290]
p2y13 = [21, 23, 26, 27, 34, 82, 85, 86, 88, 92, 95, 98, 102, 103, 106, 107, 174, 175, 196, 256, 259, 263, 277, 280, 281, 284, 288]
p2y14 = [16, 21, 29, 73, 77, 80, 81, 90, 93, 94, 101, 102, 169, 171, 174, 184, 188, 249, 253, 256, 260, 270, 274, 277, 278, 281]

# creating list of receptors
receptors = [p2y1, p2y2, p2y4, p2y6, p2y12, p2y13, p2y14]
receptors_name = ['p2y1', 'p2y2', 'p2y4', 'p2y6', 'p2y12', 'p2y13', 'p2y14']

# creating list of ligands
ligands = [ATP, ADP, AMP, adenosine, diadenosine_polyp_2, 
           diadenosine_polyp_3, diadenosine_polyp_4, diadenosine_polyp_5,
           diadenosine_polyp_6, polyP_14, cpolyP_6, polyP_40,
           polyP_60, UTP, UDP, UDP_glucose]

ligands_names = ['ATP', 'ADP', 'AMP', 'adenosine', 'diadenosine_polyp_2', 
           'diadenosine_polyp_3', 'diadenosine_polyp_4', 'diadenosine_polyp_5',
           'diadenosine_polyp_6', 'polyP_14', 'cpolyP_6', 'polyP_40',
           'polyP_60', 'UTP', 'UDP', 'UDP_glucose']

# parametrization and minimization of ligands
for smiles, lig_name in zip(ligands, ligands_names):
    if os.path.exists(f'{lig_name}.pdbqt') == False:
        file = open(f'{lig_name}.smiles', 'w')
        file.write(smiles)
        file.write('\n')
        file.close()
        os.system(f'obabel {lig_name}.smiles -O {lig_name}.mol2 --gen3d best -p 7.4 --canonical > /dev/null 2>&1')
        os.system(f'prepare_ligand4.py -l {lig_name}.mol2 -o {lig_name}.pdbqt -U nphs_lps > /dev/null 2>&1')


# DOCKING LOOP

for receptor_flex, receptor in zip(receptors, receptors_name):
    
    # parametrization of receptor
    if os.path.exists(f'{receptor}.pdbqt') == False:
        os.system(f'pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 {receptor}.pdb {receptor}.pqr > /dev/null 2>&1')
        os.system(f'prepare_receptor4.py -r {receptor}.pqr -o {receptor}.pdbqt -C -U nphs_lps -v > /dev/null 2>&1')
    
    flexible_resids_list = receptor_flex

    #calculating of receptor active site center based on flexible residues
    centers = take_loss1(f'{receptor}.pdb', [str(x) for x in flexible_resids_list])
    
    for smiles, lig_name in zip(ligands, ligands_names):
        
        dock_name = receptor + "_" + lig_name    # creating unique names for receptor+ligand complex

        # defining the size of docking grids
        if 'polyP' in lig_name:
            x_size, y_size, z_size = '25', '25', '25'
            if 'polyP_40' in lig_name or 'polyP_60' in lig_name:
                x_size, y_size, z_size = '40', '40', '40'
        else:
            x_size, y_size, z_size = '16', '16', '16'

        # docking using AutoDock VINA
        if os.path.exists(f'{dock_name}_log.txt') == False:
            write_configs(dock_name, receptor, lig_name, flexible_resids_list, centers, x_size, y_size, z_size)
        
            os.system(f'./autodock_vina_1_1_2_linux_x86/bin/vina --config {dock_name}_config_singledock --out {dock_name}_output.pdbqt --log {dock_name}_log.txt > /dev/null')

            # displaying the best VINA score for docked ligand and recptor
            out = open(f'{dock_name}_log.txt', 'r').read()
            out_split = out.splitlines()
            best_idx = out_split.index('-----+------------+----------+----------') + 1
            best_line = out_split[best_idx].split()
            best_score = best_line[1]
            print(dock_name, ':    ', best_score)


In [ ]:
# Save results in table as csv file

df = {}
df['index'] = ligands_names
for receptor in receptors_name:
    new_list = []
    for lig_name in ligands_names:
        lig_name = receptor + "_" + lig_name
        out = open(f'{lig_name}_log.txt', 'r').read()
        out_split = out.splitlines()
        print(lig_name)
        best_idx = out_split.index('-----+------------+----------+----------') + 1
        best_line = out_split[best_idx].split() # choose the best VINA score
        best_score = best_line[1] 
        new_list.append(best_score)
    df[receptor] = new_list
df = pd.DataFrame(df)
df.set_index('index', inplace=True)
df.to_csv('1_p2ys_results.csv')
